<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> Montage is an astronomical image toolkit with components for reprojection, background matching, coaddition and visualization of FITS files. It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.

The Montage source is written in ANSI-C and code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ). The Python package can be installed from PyPI ("</i>pip install MontagePy<i>"). The package has no external dependencies. See http://montage.ipac.caltech.edu/ for details on the design and applications of Montage.


# MontagePy.main modules: mTANHdr

mTANHdr is a special utility that extends the range of the mProjectPP reprojection module.  mProjectPP is much faster than the more general mProject but is limited to tangent-plane projections (<i>e.g.</i>, TAN, SIN). It does, however, support focal-plane distortion parameters (<i>e.g.</i>, TAN-SIP) so there is a trick we can use that lets is work with a range of other projections.

Many images we deal with in astronomy cover a fairly small area on the sky and in those areas it is often the case that many projections (<i>e.g.</i>, cylindrical CAR) can be matched with a distorted TAN to very small fractions of a pixel and the extra time taken in fitting for the distortion parameter values is more that made up for by the speed-up in using mProjectPP for the reprojection.

In [1]:
from MontagePy.main import mTANHdr, mViewer

help(mTANHdr)

Help on built-in function mTANHdr in module MontagePy.main:

mTANHdr(...)
    mTANHdr attempts to create a distorted-TAN header that matches the original image header (to speed reprojection in mProjectPP).
    
    Parameters
    ----------
    template : str
        Original image file FITS header.
    tanTemplate : str
        Output matching distorted-TAN header.
    order : int, optional
        Polynomial order for distortions.
    maxiter : int, optional
        Maximum number of iterations for fitting.
    tolerance : float, optional
        Tolerance (fractional pixel maximum error) for fit.
    useOffscl : bool, optional
        Include pixels that are just off the image in fitting.
    debug : int, optional
        Debugging output flag.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).
    fwdxerr : float
        Maximum pixel error for forward transform in X axis.
    fwdyerr : float

## Example

One of the products in these examples is a mosaic of a one-degree square area around M17.  This is normally done in TAN (gnomonic) projection but we can also so it in the simple cylindrical CAR projection, then reproject this mosaic to TAN either with mProject or with mProjectPP using a distorted TAN header made by mTANHdr:

In [2]:
rtn = mTANHdr("M17/mosaic_car.hdr", "work/M17/distorted.hdr")

print(rtn)

{'status': '0', 'fwdxerr': 0.003812273559105961, 'fwdyerr': 0.009351456828881055, 'fwditer': 13, 'revxerr': 0.003837209781522688, 'revyerr': 0.00967726188900997, 'reviter': 13}


The error values in the return show the worst-case positional difference between the distorted TAN and original CAR projections, in pixels.  In this case, since the pixels are 1 arcsecond, the maximum positional difference is around 10 milli-arcseconds.

Reprojecting the CAR image with mProject takes about 280 seconds.  Using mProjectPP and this distorted TAN header takes only 12 seconds; a factor of 23 faster.

Here is the distorted TAN header that mTANHdr produced:

In [3]:
with open("work/M17/distorted.hdr", "r") as file:
    for line in file:  
        print(line, end='')

SIMPLE  = T
BITPIX  = -64
NAXIS   = 2
NAXIS1  = 3600
NAXIS2  = 3600
CTYPE1  = 'RA---TAN-SIP'
CTYPE2  = 'DEC--TAN-SIP'
CRVAL1  =  275.1962900000
CRVAL2  =  -16.1715300000
CRPIX1  = 1800.5000000000
CRPIX2  = 1800.5000000000
CDELT1  = -0.0002777784
CDELT2  = 0.0002777758
CROTA2  = 0.000000
EQUINOX = 2000.0
A_ORDER = 4
A_0_0   = -2.956e-03
A_0_1   = -1.836e-11
A_0_2   = -1.832e-14
A_0_3   =  8.875e-19
A_0_4   =  4.903e-24
A_1_0   = -4.546e-07
A_1_1   = -6.517e-14
A_1_2   =  8.148e-18
A_1_3   =  3.317e-22
A_1_4   = -2.100e-25
A_2_0   = -2.177e-11
A_2_1   =  9.580e-18
A_2_2   =  7.656e-22
A_2_3   = -1.784e-24
A_2_4   =  2.277e-28
A_3_0   =  7.835e-12
A_3_1   =  1.009e-22
A_3_2   = -1.723e-24
A_3_3   =  6.697e-28
A_3_4   = -8.159e-32
A_4_0   = -3.595e-21
A_4_1   = -1.894e-25
A_4_2   =  2.112e-28
A_4_3   = -7.775e-32
A_4_4   =  8.966e-36
B_ORDER = 4
B_0_0   = -7.397e-03
B_0_1   =  3.992e-06
B_0_2   = -2.249e-11
B_0_3   =  7.834e-12
B_0_4   = -3.489e-21
B_1_0   = -4.590e-07
B_1_1   = -1.998e-11

&nbsp;<p/>

## Error Handling

If mTANHdr encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user specifies a header file that doesn't exist: 

In [5]:
rtn = mTANHdr("input/M17/unknown.hdr", "work/M17/distorted.hdr")

print(rtn)

{'status': '1', 'msg': b'Bad template: input/M17/unknown.hdr'}


&nbsp;

## Classic Montage: mTANHdr as a Stand-Alone Program

### mTANHdr Unix/Windows Command-line Arguments

<p>mTANHdr can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> No child program executable or program aborted
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mTANHdr can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mTANHdr                                                              */
/*                                                                       */
/*  Montage is a set of general reprojection / coordinate-transform /    */
/*  mosaicking programs.  Any number of input images can be merged into  */
/*  an output FITS file.  The attributes of the input are read from the  */
/*  input files; the attributes of the output are read a combination of  */
/*  the command line and a FITS header template file.                    */
/*                                                                       */
/*  There are two main reprojection modules, mProject and mProjectPP.    */
/*  The first can handle any projection transformation but is slow.  The */
/*  second can only handle transforms between tangent plane projections  */
/*  but is very fast.  In many cases, however, a non-tangent-plane       */
/*  projection can be approximated by a TAN projection with pixel-space  */
/*  distortions (in particular when the region covered is small, which   */
/*  is often the case in practice).                                      */
/*                                                                       */
/*  This module analyzes a template file and determines if there is      */
/*  an adequate equivalent distorted TAN projection that would be        */
/*  equivelent (i.e. location shifts less than, say, 0.01 pixels).       */
/*  mProjectPP can then be used to produce this distorted TAN image      */
/*  with the original non-TAN FITS header swapped in before writing      */
/*  to disk.                                                             */
/*                                                                       */
/*                                                                       */
/*  NOTE:  The 'reverse' error is the important one for deciding whether */
/*  the new distorted-TAN header can be used in place of the original    */
/*  when reprojecting in mProjectPP since it is a measure of the         */
/*  process of going from distorted TAN to sky to original projection.   */
/*  Since the second part of this is exact, this error is all about how  */
/*  accurately the distorted-TAN maps to the right point of the sky.     */
/*                                                                       */
/*   char  *origtmpl       Original image file FITS header               */
/*   char  *newtmpl        Output matching distorted-TAN header          */
/*   int    order          Polynomial order for distortions              */
/*   int    maxiter        Maximum number of iterations for fitting      */
/*   int    tolerance      Tolerance (maximum error) for fit             */
/*   int    useOffscl      Include in the fitting pixels that are        */
/*                         just off the image                            */
/*                                                                       */
/*   int    debug          Debugging output flag                         */
/*                                                                       */
/*************************************************************************/

struct mTANHdrReturn *mTANHdr(char *origtmpl, char *newtmpl, int order, int maxiter, double tolerance, 
                              int useOffscl, int debugin)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mTANHdrReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
   double fwdxerr;       // Maximum error in X for forward transformation
   double fwdyerr;       // Maximum error in Y for forward transformation
   double fwditer;       // Number of iterations needed in forward transformation
   double revxerr;       // Maximum error in X for reverse transformation
   double revyerr;       // Maximum error in Y for reverse transformation
   double reviter;       // Number of iterations needed in reverse transformation
};
</pre>